
<hr style="height:2px;border:none"/>
<H1 align='center'> Challenge Kaggle </H1>
<H5 align='center'> Predicción del precio de una casa: Solución</H5>

Lenguaje: Python

Temas:

    - Arquitectura de Redes Neuronales Feed-Forward
    - Entrenamiento de Redes Neuronales. 
    - Parte Básica de Redes Convolucionales. 
    - Problemas Especiales.

### Contexto

Queremos predecir el precio de una casa a partir de metadata como el número de habitaciones, el número de baños, el área total de la casa y el zipcode.

### Dataset

El dataset de entrenamiento incluye la información de 443 propiedades con sus respectivos precios. El dataset de pruebas incluye la información de 92 propiedades, pero no estarán disponibles los precios.

**La métrica de evaluación para esta competencia es MSE** 

In [1]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

def root_mean_squared_error(y_true, y_pred):  # loss rmse
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))
path = './data/'

/Users/panchoandrades/.pyenv/versions/3.10/envs/auxx/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#se leen los datos
df_train = pd.read_csv(path+'metadata_casas_train.csv',index_col=0)
df_test = pd.read_csv(path+'metadata_casas_test.csv',index_col=0)
df_train.head()

,cod_casa,habitaciones,banos,area,zipcode,precio
0,OC4U9J6C11R5715,4,4.0,4053,85255,869500
1,CQF44W2LE3S3NSC,4,3.0,3343,36372,865200
2,LT2Z9MOPF5KIM7T,3,4.0,3923,85266,889000
3,VUCNPXL2QMIA22A,5,5.0,4022,85262,910000
4,9CWP6T0SBBLGDBL,3,4.0,4116,85266,971226


In [3]:
#para testing
df_train,df_test = train_test_split(df_train,test_size=0.25,
                                    shuffle=True,random_state=0)

In [4]:
# se realiza la codificación de los zipcodes mencionada en el notebook EDA
def process_metadata(df,lista_columnas,flag):
    #esto debe poder hacerse de mejor manera, falta un refactor.
    df_new = df.copy()
    
    codes = df_new.zipcode
    for n_digits in range(3):
        new_codes = [int(str(i)[:(2*n_digits)+1]) for i in codes.values]
        df_new[str(n_digits)+'_codes'] = new_codes
        one_hot = pd.get_dummies(df_new[str(n_digits)+'_codes'],prefix=str(n_digits))
        df_new = df_new.join(one_hot)
        df_new.drop(str(n_digits)+'_codes',axis=1,inplace=True)
    df_new.drop('zipcode',axis=1,inplace=True)
    if flag: #nos quedamos solo con las columnas que estaban en train
        columnas_test = []
        for columna in df_new.columns:
            if columna in lista_columnas:
                columnas_test.append(columna)
        df_new = df_new.loc[:,columnas_test]
        df_empty = pd.DataFrame(columns = lista_columnas)
        df_new = pd.concat([df_empty, df_new], axis=0).fillna(0)
    
    return df_new,df_new.columns

df_train_clean1,lista_columnas = process_metadata(df_train,[],0)
df_test_clean1,_ = process_metadata(df_test,lista_columnas,1)

/var/folders/tl/bbz8s4bj59zdwgn7k57_mc2w0000gn/T/ipykernel_73604/3054942506.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new = pd.concat([df_empty, df_new], axis=0).fillna(0)
/var/folders/tl/bbz8s4bj59zdwgn7k57_mc2w0000gn/T/ipykernel_73604/3054942506.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new = pd.concat([df_empty, df_new], axis=0).fillna(0)


In [5]:
# se elimina la variable cod_casa que no tiene utilidad (es un id)
df_train_clean1.drop('cod_casa',axis=1,inplace=True)
df_test_clean1.drop('cod_casa',axis=1,inplace=True)

In [6]:
# No se le hace ninguna transformación al precio. Eliminar los outliers más severos resultó
# en una mejor performance en nuestros datos de testing, pero peor en la competencia. Esto
# no es ninguna sorpresa, simplemente quiere decir que existen datos en la competencia que 
# están relacionados con los outliers más severos.

# Trabajar realizando una transformación logarítmica o un reemplazo con algún valor estratégico
# (media/mediana/valor_máximo) resultó en un pero desempeño.
y_train = df_train_clean1['precio']
y_test = df_test_clean1['precio']

X_train = df_train_clean1.drop(['precio'],axis=1)
X_test = df_test_clean1.drop(['precio'],axis=1)

In [7]:
# Se escalan los datos.
scaler_data = StandardScaler()
X_train.iloc[:,:3] = X_train.iloc[:,:3].astype('float64')
X_train.iloc[:,:3] = scaler_data.fit_transform(X_train.iloc[:,:3])
X_test.iloc[:,:3] = X_test.iloc[:,:3].astype('float64')
X_test.iloc[:,:3] = scaler_data.transform(X_test.iloc[:,:3])

scaler_precio = StandardScaler()
y_train = scaler_precio.fit_transform(np.array(y_train).reshape(-1,1))
y_test = scaler_precio.transform(np.array(y_test).reshape(-1,1))

/var/folders/tl/bbz8s4bj59zdwgn7k57_mc2w0000gn/T/ipykernel_73604/1983760836.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '23     5.0
422    3.0
341    2.0
15     4.0
363    2.0
      ... 
387    2.0
238    5.0
147    3.0
55     3.0
216    5.0
Name: habitaciones, Length: 332, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_train.iloc[:,:3] = X_train.iloc[:,:3].astype('float64')
/var/folders/tl/bbz8s4bj59zdwgn7k57_mc2w0000gn/T/ipykernel_73604/1983760836.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '23     5677.0
422    2346.0
341    1344.0
15     3721.0
363    1248.0
        ...  
387    1536.0
238    3200.0
147    2125.0
55     2836.0
216    5000.0
Name: area, Length: 332, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

In [8]:
# Se define modelo Feed-Forward densa con sólo 1 capa oculta con 4 neuronas
# Este modelo se obtuvo mediante grid search  y tuning
# Es el modelo que mejor se comportó 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class ModeloMetadata(keras.Model):
    def __init__(self, dim_hidden):
        super(ModeloMetadata, self).__init__(name='Modelo_metadata')
        self.dense1 = layers.Dense(dim_hidden, activation='relu')
        self.output_layer = layers.Dense(1, name='Output')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.output_layer(x)

# Uso
model = ModeloMetadata(dim_hidden = 4)
dummy_input = tf.zeros((1, X_train.shape[1]))
model(dummy_input) 
model.summary()

Model: "Modelo_metadata"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (1, 4)                 │           292 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (1, 1)                 │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 297 (1.16 KB)

 Trainable params: 297 (1.16 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Se entrena el modelo por 200 epochs.
model.compile(loss=root_mean_squared_error)
history = model.fit(
        x = X_train,
        y = y_train,
        epochs=200,
        verbose=1,
    ) 
preds = model.predict(X_test)

Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - loss: 1.0026
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - loss: 0.8175
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.8260
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - loss: 0.7890
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - loss: 0.8050
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - loss: 0.8726
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.9789
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - loss: 0.7895
Epoch 9/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: 0.9537
Epoch 10/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: 0.7280
Epoch 11/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - loss: 0.7965
Epoch 12/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - loss: 0.8694
Epoch 13/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - loss: 0.9663
Epoch 14/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - loss: 0.6189
Epoch 15/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [10]:
#para testing
#Error de 300.000 aprox en un rango de valores [0,6.000.000]. Es un error relativamente pequeño.
#Los outliers hacen mucho daño, eliminando los más severos el error disminuye considerablemente.
rmse(scaler_precio.inverse_transform(y_test),scaler_precio.inverse_transform(preds))
#rmse(y_test,preds)

302122.1537053342

In [11]:
#Ignorar, esto es para la competencia
#submission = pd.read_csv('../input/sample-submission/sampleSubmission.csv')
#submission['\'value\''] = scaler_precio.inverse_transform(preds)
#submission.to_csv('submission.csv',index=False)
#submission